<a href="https://colab.research.google.com/github/rainardw/Moonlight-Diary/blob/main/Moonlight_Diary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# INSTALL (cukup sekali)
!pip install ipywidgets

import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import hashlib
import json
import os
import datetime
import getpass  # untuk input password tersembunyi

In [12]:
diary_css = """
<style>
@import url('https://fonts.googleapis.com/css2?family=Caveat:wght@600;700&family=Dancing+Script:wght@700&family=Poppins:wght@300;400;500&display=swap');

/* WADAH UTAMA */
.diary-book {
    background: linear-gradient(135deg, #fff0f5 0%, #ffe4e9 100%);
    padding: 30px;
    border-radius: 25px;
    color: #5a3d5c;
    max-width: 900px;
    margin: 20px auto;
    box-shadow: 0 10px 40px rgba(255, 182, 193, 0.3);
    position: relative;
    overflow: hidden;
    border: 1px solid rgba(255, 182, 193, 0.2);
}

/* COVER BUKU */
.book-cover {
    background: linear-gradient(135deg, #ffb6c1, #ffccd5, #ffebf0);
    padding: 25px;
    border-radius: 15px 15px 0 0;
    text-align: center;
    position: relative;
    z-index: 10;
    box-shadow: 0 5px 15px rgba(255, 182, 193, 0.3);
}

.book-title {
    font-family: 'Dancing Script', cursive;
    font-size: 3.2em;
    color: #ffffff;
    text-shadow: 2px 2px 8px rgba(155, 89, 182, 0.4);
    letter-spacing: 1.5px;
    margin-bottom: 5px;
}

.book-subtitle {
    font-family: 'Poppins', sans-serif;
    font-size: 1.1em;
    color: rgba(255, 255, 255, 0.9);
    font-weight: 300;
    margin-bottom: 15px;
    text-shadow: 1px 1px 4px rgba(155, 89, 182, 0.3);
}

/* HALAMAN BUKU */
.book-page {
    background: linear-gradient(to bottom,
        rgba(255, 255, 255, 0.95) 0%,
        rgba(255, 250, 250, 0.95) 100%);
    color: #5a3d5c;
    padding: 35px;
    border-radius: 0 0 15px 15px;
    min-height: 300px;
    font-family: 'Poppins', sans-serif;
    font-size: 16px;
    line-height: 1.7;
    box-shadow: inset 0 0 10px rgba(255, 182, 193, 0.1);
    border: 1px solid rgba(255, 182, 193, 0.1);
}

/* LOCK SCREEN */
.lock-screen {
    position: absolute;
    top: 0; left: 0; right: 0; bottom: 0;
    background: rgba(255, 255, 255, 0.92);
    display: flex;
    flex-direction: column;
    justify-content: center;
    align-items: center;
    z-index: 100;
    border-radius: 15px;
    backdrop-filter: blur(8px);
}

.lock-icon {
    font-size: 5em;
    color: #ff9aa2;
    margin-bottom: 20px;
    animation: float 3s infinite ease-in-out;
}

@keyframes float {
    0%, 100% { transform: translateY(0); }
    50% { transform: translateY(-10px); }
}

.lock-text {
    font-family: 'Dancing Script', cursive;
    font-size: 2em;
    color: #9b59b6;
    text-align: center;
    margin: 10px 0;
}

.lock-hint {
    font-family: 'Poppins', sans-serif;
    font-size: 0.9em;
    color: #95a5a6;
    font-style: italic;
}

/* TEXT AREA */
.write-area {
    width: 100%;
    height: 250px;
    padding: 20px;
    border: 2px solid #ffdde1;
    border-radius: 15px;
    font-family: 'Poppins', sans-serif;
    font-size: 16px;
    line-height: 1.7;
    resize: vertical;
    background: rgba(255, 255, 255, 0.9);
    margin: 20px 0;
    color: #5a3d5c;
    box-shadow: 0 4px 15px rgba(255, 182, 193, 0.1);
    transition: all 0.3s ease;
}

.write-area:focus {
    outline: none;
    border-color: #ff9aa2;
    box-shadow: 0 0 20px rgba(255, 154, 162, 0.3);
    background: white;
}

/* TOMBOL */
.book-btn {
    margin: 8px;
    padding: 14px 28px;
    border: none;
    border-radius: 50px;
    font-weight: 500;
    cursor: pointer;
    transition: all 0.3s ease;
    color: white;
    font-size: 16px;
    font-family: 'Poppins', sans-serif;
    min-width: 160px;
    letter-spacing: 0.5px;
    box-shadow: 0 4px 12px rgba(0,0,0,0.1);
}

.book-btn:hover {
    transform: translateY(-3px);
    box-shadow: 0 8px 20px rgba(0,0,0,0.15);
}

.btn-write {
    background: linear-gradient(45deg, #ff9aa2, #ffb7b2);
    border-left: 4px solid #ff6b8b;
}
.btn-save {
    background: linear-gradient(45deg, #a29bfe, #74b9ff);
    border-left: 4px solid #6c5ce7;
}
.btn-view {
    background: linear-gradient(45deg, #fd79a8, #e84393);
    border-left: 4px solid #d63031;
}
.btn-lock {
    background: linear-gradient(45deg, #00cec9, #00b894);
    border-left: 4px solid #00a085;
}

/* PASSWORD INPUT */
.pwd-input {
    padding: 14px 20px;
    border: 2px solid #dfe6e9;
    border-radius: 50px;
    font-family: 'Poppins', sans-serif;
    font-size: 15px;
    width: 320px;
    margin-right: 10px;
    transition: all 0.3s ease;
    color: #5a3d5c;
}

.pwd-input:focus {
    outline: none;
    border-color: #a29bfe;
    box-shadow: 0 0 15px rgba(162, 155, 254, 0.2);
}

/* MESSAGE BOX */
.msg-box {
    text-align: center;
    padding: 15px;
    margin: 20px 0;
    border-radius: 12px;
    font-weight: 500;
    font-family: 'Poppins', sans-serif;
    border: 1px solid;
}
.msg-success {
    background: rgba(116, 185, 255, 0.15);
    color: #0984e3;
    border-color: rgba(116, 185, 255, 0.3);
}
.msg-error {
    background: rgba(255, 107, 107, 0.15);
    color: #d63031;
    border-color: rgba(255, 107, 107, 0.3);
}
.msg-info {
    background: rgba(255, 184, 184, 0.15);
    color: #e84393;
    border-color: rgba(255, 184, 184, 0.3);
}

/* ENTRY LIST */
.entry-list {
    background: rgba(255, 255, 255, 0.95);
    border: 1px solid #ffdde1;
    border-radius: 15px;
    padding: 25px;
    margin-top: 20px;
    max-height: 400px;
    overflow-y: auto;
    font-family: 'Poppins', sans-serif;
    box-shadow: 0 8px 25px rgba(255, 182, 193, 0.15);
}

.entry-item {
    padding: 18px 0;
    border-bottom: 1px dashed #ffdde1;
    transition: all 0.3s ease;
}
.entry-item:hover {
    background: rgba(255, 235, 240, 0.5);
    padding-left: 10px;
}
.entry-item:last-child {
    border-bottom: none;
}
.entry-date {
    font-weight: 600;
    color: #9b59b6;
    font-size: 0.95em;
    margin-bottom: 5px;
}
.entry-preview {
    color: #5a3d5c;
    font-style: italic;
    line-height: 1.5;
}

/* MODAL */
.modal-overlay {
    position: fixed;
    top: 0;
    left: 0;
    right: 0;
    bottom: 0;
    background: rgba(0, 0, 0, 0.7);
    display: flex;
    justify-content: center;
    align-items: center;
    z-index: 9999;
    backdrop-filter: blur(5px);
}

.modal-content {
    background: white;
    padding: 30px;
    border-radius: 20px;
    max-width: 700px;
    max-height: 80vh;
    overflow-y: auto;
    box-shadow: 0 25px 50px rgba(0,0,0,0.3);
    font-family: 'Poppins', sans-serif;
    color: #5a3d5c;
}

.modal-header {
    display: flex;
    justify-content: space-between;
    align-items: center;
    margin-bottom: 25px;
    padding-bottom: 15px;
    border-bottom: 2px solid #ffdde1;
}

.modal-title {
    font-family: 'Dancing Script', cursive;
    font-size: 2.2em;
    color: #9b59b6;
    margin: 0;
}

.modal-close {
    background: #ff6b6b;
    color: white;
    border: none;
    border-radius: 50%;
    width: 36px;
    height: 36px;
    cursor: pointer;
    font-size: 20px;
    transition: all 0.3s ease;
}
.modal-close:hover {
    background: #ff4757;
    transform: rotate(90deg);
}

.modal-body {
    font-size: 16px;
    line-height: 1.8;
    white-space: pre-wrap;
    padding: 20px;
    background: #f9f9f9;
    border-radius: 12px;
    border: 1px solid #ffdde1;
}
</style>
"""

display(HTML(diary_css))

# === STATE ===
DIARY_FILE = "my_diary.json"
password_hash = None
entries = []
unlocked = False

# === FUNGSI UTAMA ===
def load_diary():
    global password_hash, entries
    if os.path.exists(DIARY_FILE):
        try:
            with open(DIARY_FILE, 'r', encoding='utf-8') as f:
                data = json.load(f)
                password_hash = data.get("password_hash")
                entries = data.get("entries", [])
        except Exception as e:
            print(f"Error loading diary: {e}")
            password_hash = None
            entries = []
    else:
        password_hash = None
        entries = []

def save_diary():
    try:
        with open(DIARY_FILE, 'w', encoding='utf-8') as f:
            json.dump({
                "password_hash": password_hash,
                "entries": entries
            }, f, indent=2, ensure_ascii=False)
    except Exception as e:
        print(f"Error saving diary: {e}")

def set_password(pwd):
    global password_hash
    password_hash = hashlib.sha256(pwd.encode()).hexdigest()
    save_diary()

def verify_password(pwd):
    if not password_hash:
        return False
    return hashlib.sha256(pwd.encode()).hexdigest() == password_hash

# === WIDGETS ===
cover = widgets.HTML('''
<div class="book-cover">
    <div class="book-title">🌸 Moonlight Journal</div>
    <div class="book-subtitle">Tulis rahasia hatimu di atas kertas ini.</div>
</div>
''')

page = widgets.HTML('')
lock_screen = widgets.HTML('')

# Text area untuk menulis
write_area = widgets.Textarea(
    placeholder="✨ Tulis perasaanmu di sini...\nBiarkan kata-kata mengalir dengan bebas.\n\n💭 Tips: Tulis apa yang kamu rasakan, pikirkan, atau impikan hari ini.",
    layout=widgets.Layout(width='100%', height='250px', margin='20px 0')
)
write_area.disabled = True

# Tombol-tombol
write_btn = widgets.Button(description='📝 Tulis Baru', layout=widgets.Layout(width='160px'))
write_btn.add_class('book-btn btn-write')

save_btn = widgets.Button(description='💾 Simpan', layout=widgets.Layout(width='160px'))
save_btn.add_class('book-btn btn-save')

view_btn = widgets.Button(description='👁️ Lihat Diary', layout=widgets.Layout(width='160px'))
view_btn.add_class('book-btn btn-view')

lock_btn = widgets.Button(description='🔓 Buka Kunci', layout=widgets.Layout(width='160px'))
lock_btn.add_class('book-btn btn-lock')

# Input password dengan style baru
pwd_input = widgets.Password(
    value='',
    placeholder='Masukkan password baru/untuk buka',
    layout=widgets.Layout(width='320px', margin='0 10px 0 0')
)
pwd_input.add_class('pwd-input')

pwd_submit = widgets.Button(description='🔓 Submit', layout=widgets.Layout(width='100px'))
pwd_submit.add_class('book-btn')
pwd_submit.style.button_color = '#a29bfe'

output_area = widgets.Output()

showing_password_input = False

def render_book():
    global unlocked
    if unlocked:
        today = datetime.datetime.now().strftime("%A, %d %B %Y")
        page.value = f'''
        <div class="book-page">
            <div style="text-align:center; color:#9b59b6; font-family:'Dancing Script', cursive; font-size:2em; margin-bottom:20px;">
                ✨ {today} ✨
            </div>
            <div style="color: #74b9ff; text-align: center; margin-bottom: 25px; font-family:'Poppins', sans-serif;">
                "Jiwamu adalah taman rahasia, biarkan bunga-bunga kata tumbuh di sini"
            </div>
            <div style="color: #636e72; text-align: center; font-size: 0.9em; font-family:'Poppins', sans-serif;">
                📖 {len(entries)} entri tersimpan
            </div>
        </div>
        '''
        lock_screen.value = ""
        write_area.disabled = False
        lock_btn.description = '🔐 Kunci'
        lock_btn.style.button_color = '#00cec9'
        pwd_input.layout.display = 'none'
        pwd_submit.layout.display = 'none'
    else:
        lock_screen.value = '''
        <div class="lock-screen">
            <div class="lock-icon">🔒</div>
            <div class="lock-text">Jurnal Rahasia Terkunci</div>
            <div class="lock-hint">Masukkan password untuk membuka dunia hatimu</div>
        </div>
        '''
        page.value = '<div class="book-page"></div>'
        write_area.disabled = True
        lock_btn.description = '🔓 Buka Kunci'
        lock_btn.style.button_color = '#00cec9'
        if showing_password_input:
            pwd_input.layout.display = 'flex'
            pwd_submit.layout.display = 'flex'
        else:
            pwd_input.layout.display = 'none'
            pwd_submit.layout.display = 'none'

def show_message(msg, msg_type="info"):
    if msg_type == "success":
        cls = "msg-success"
    elif msg_type == "error":
        cls = "msg-error"
    else:
        cls = "msg-info"

    with output_area:
        clear_output()
        display(HTML(f'<div class="msg-box {cls}">✨ {msg} ✨</div>'))

def on_lock_click(b):
    global unlocked, showing_password_input
    load_diary()
    if unlocked:
        unlocked = False
        write_area.value = ""
        render_book()
        show_message("Jurnal telah dikunci dengan aman", "success")
        showing_password_input = False
    else:
        pwd_input.value = ''
        showing_password_input = True
        render_book()

def on_pwd_submit_click(b):
    global unlocked
    pwd = pwd_input.value.strip()
    if not pwd:
        show_message("Password tidak boleh kosong", "error")
        return

    if not password_hash:
        if len(pwd) < 4:
            show_message("Password minimal 4 karakter", "error")
            return
        set_password(pwd)
        unlocked = True
        render_book()
        show_message("🎉 Password berhasil dibuat! Selamat menulis di diary-mu", "success")
    else:
        if verify_password(pwd):
            unlocked = True
            render_book()
            show_message("🔓 Selamat datang kembali! Dunia hatimu menunggu", "success")
        else:
            show_message("❌ Password salah. Coba lagi!", "error")

def on_write_click(b):
    if not unlocked:
        show_message("🔒 Buka kunci jurnal dulu!", "error")
        return

    write_area.value = ""
    show_message("📝 Siap menulis! Isi diary-mu di bawah ini", "info")

def on_save_click(b):
    global unlocked
    if not unlocked:
        show_message("🔒 Buka kunci jurnal dulu!", "error")
        return

    content = write_area.value.strip()
    if not content:
        show_message("Tulis sesuatu dulu, sayang", "error")
        return

    entry = {
        "date": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "title": f"Entri #{len(entries) + 1}",
        "content": content
    }

    entries.append(entry)
    save_diary()
    write_area.value = ""
    show_message(f"✅ Diary tersimpan! Total entri: {len(entries)}", "success")
    render_book()

def on_view_click(b):
    global unlocked
    if not unlocked:
        show_message("🔒 Buka kunci jurnal dulu!", "error")
        return

    if not entries:
        show_message("📭 Diary masih kosong. Mulailah menulis!", "info")
        return

    html = '''
    <div class="entry-list">
        <h3 style="color: #9b59b6; text-align: center; font-family: \'Dancing Script\', cursive; font-size: 2em;">📚 Semua Entri Diary</h3>
        <hr style="border: none; height: 2px; background: linear-gradient(to right, transparent, #ffdde1, transparent); margin: 20px 0;">
    '''

    for i, entry in enumerate(reversed(entries)):
        date = entry["date"]
        content = entry["content"]
        preview = content[:120] + "..." if len(content) > 120 else content

        html += f'''
        <div class="entry-item">
            <div class="entry-date">📅 {date}</div>
            <div class="entry-preview">"{preview}"</div>
            <button onclick="showEntryModal(`{content.replace('`', "'").replace('\\n', '<br>')}`, `{date}`)"
                    style="background: linear-gradient(45deg, #fd79a8, #e84393); color: white; border: none; padding: 8px 16px; border-radius: 25px; margin-top: 10px; cursor: pointer; font-family: \'Poppins\', sans-serif;">
                📖 Baca Lengkap
            </button>
        </div>
        '''

    html += '</div>'

    with output_area:
        clear_output()
        display(HTML(html))

# === HUBUNGKAN TOMBOL ===
pwd_submit.on_click(on_pwd_submit_click)
write_btn.on_click(on_write_click)
save_btn.on_click(on_save_click)
view_btn.on_click(on_view_click)
lock_btn.on_click(on_lock_click)

# === RENDER AWAL ===
load_diary()
render_book()

# === TAMPILKAN SEMUA ===
button_row = widgets.HBox([
    write_btn,
    save_btn,
    view_btn,
    lock_btn
], layout=widgets.Layout(
    justify_content='center',
    margin='20px 0',
    flex_wrap='wrap',
    align_items='center'
))

password_row = widgets.HBox([
    pwd_input,
    pwd_submit
], layout=widgets.Layout(
    justify_content='center',
    margin='15px 0'
))

ui = widgets.VBox([
    cover,
    widgets.HTML('<div style="position:relative;">'),
    page,
    lock_screen,
    widgets.HTML('</div>'),
    write_area,
    button_row,
    password_row,
    output_area
], layout=widgets.Layout(align_items='center'))

display(ui)

# =============================================
# 🌸 EFEK PETAL BUNGA YANG WORK 100% 🌸
# =============================================
petal_effects = """
<div id="petalContainer" style="
    position: fixed;
    top: 0;
    left: 0;
    width: 100%;
    height: 100%;
    pointer-events: none;
    z-index: 9998;
    overflow: hidden;
"></div>

<script>
// Array bunga yang berbeda
const petals = ['🌸', '🌺', '🌼', '🌷', '🌹', '💮', '🏵️', '🥀', '🌻', '🌾'];
const colors = ['#FF9AA2', '#FFB7B2', '#FFDAC1', '#E2F0CB', '#B5EAD7', '#C7CEEA', '#FFB5E8', '#FFCCF9', '#A0E7E5', '#B5DEFF'];

// Fungsi untuk membuat satu petal
function createPetal() {
    const petal = document.createElement('div');
    petal.style.position = 'absolute';
    petal.style.fontSize = (Math.random() * 30 + 20) + 'px';
    petal.style.opacity = Math.random() * 0.6 + 0.2;
    petal.style.color = colors[Math.floor(Math.random() * colors.length)];
    petal.style.textShadow = '0 2px 10px rgba(255,255,255,0.3)';
    petal.innerHTML = petals[Math.floor(Math.random() * petals.length)];

    // Posisi awal random di atas layar
    petal.style.left = Math.random() * 100 + 'vw';
    petal.style.top = '-50px';

    // Animasi jatuh dengan variasi
    const duration = Math.random() * 15 + 10;
    const delay = Math.random() * 5;
    const sway = Math.random() * 200 - 100;

    // Keyframes untuk animasi
    const keyframes = `
        @keyframes fall-${Date.now()}-${Math.random().toString(36).substr(2, 9)} {
            0% {
                transform: translate(0, 0) rotate(0deg);
                opacity: ${petal.style.opacity};
            }
            25% {
                transform: translate(${sway * 0.3}px, 25vh) rotate(90deg);
            }
            50% {
                transform: translate(${sway * 0.6}px, 50vh) rotate(180deg);
            }
            75% {
                transform: translate(${sway * 0.3}px, 75vh) rotate(270deg);
            }
            100% {
                transform: translate(0, 100vh) rotate(360deg);
                opacity: 0;
            }
        }
    `;

    // Tambahkan style ke document
    const style = document.createElement('style');
    style.textContent = keyframes;
    document.head.appendChild(style);

    // Ambil nama animasi dari keyframes
    const animationName = keyframes.match(/@keyframes (\S+)/)[1];

    // Terapkan animasi
    petal.style.animation = `${animationName} ${duration}s linear ${delay}s infinite`;

    return petal;
}

// Fungsi untuk memulai efek petal
function startPetalEffect() {
    const container = document.getElementById('petalContainer');
    const petalCount = 30;

    // Buat petal pertama
    for (let i = 0; i < petalCount; i++) {
        setTimeout(() => {
            const petal = createPetal();
            container.appendChild(petal);

            // Hapus dan buat ulang petal setelah selesai animasi
            setTimeout(() => {
                if (petal.parentNode) {
                    petal.remove();
                }
                const newPetal = createPetal();
                container.appendChild(newPetal);
            }, (duration + delay) * 1000);

        }, i * 300);
    }
}

// Jalankan efek petal setelah halaman dimuat
document.addEventListener('DOMContentLoaded', startPetalEffect);

// Juga jalankan sekarang (untuk jaga-jaga)
startPetalEffect();

// Fungsi untuk modal
function showEntryModal(content, date) {
    // Hapus modal yang sudah ada
    const existingModal = document.getElementById('diaryModal');
    if (existingModal) existingModal.remove();

    // Buat modal baru
    const modalHTML = `
        <div class="modal-overlay" id="diaryModal">
            <div class="modal-content">
                <div class="modal-header">
                    <h2 class="modal-title">📖 Diary Entry</h2>
                    <button class="modal-close" onclick="document.getElementById('diaryModal').remove()">×</button>
                </div>
                <div class="modal-body">
                    <h4 style="color: #9b59b6; margin-bottom: 15px;">${date}</h4>
                    <div style="white-space: pre-wrap; line-height: 1.8;">${content}</div>
                </div>
            </div>
        </div>
    `;

    document.body.insertAdjacentHTML('beforeend', modalHTML);
}

// Fungsi untuk efek hover pada petal
document.addEventListener('mousemove', function(e) {
    const petalContainer = document.getElementById('petalContainer');
    if (!petalContainer) return;

    const petals = petalContainer.children;
    const mouseX = e.clientX;
    const mouseY = e.clientY;

    for (let petal of petals) {
        const rect = petal.getBoundingClientRect();
        const petalCenterX = rect.left + rect.width / 2;
        const petalCenterY = rect.top + rect.height / 2;

        const distance = Math.sqrt(
            Math.pow(mouseX - petalCenterX, 2) +
            Math.pow(mouseY - petalCenterY, 2)
        );

        if (distance < 80) {
            petal.style.transform += ' scale(1.3)';
            petal.style.filter = 'brightness(1.5) drop-shadow(0 0 8px rgba(255,255,255,0.5))';
            petal.style.zIndex = '9999';

            setTimeout(() => {
                petal.style.transform = petal.style.transform.replace(' scale(1.3)', '');
                petal.style.filter = '';
                petal.style.zIndex = '';
            }, 300);
        }
    }
});

// Refresh petal setiap 30 detik untuk mencegah lag
setInterval(() => {
    const container = document.getElementById('petalContainer');
    if (container.children.length < 20) {
        for (let i = 0; i < 5; i++) {
            const petal = createPetal();
            container.appendChild(petal);
        }
    }
}, 30000);
</script>

<style>
/* Animasi tambahan untuk efek berkilau */
@keyframes sparkle {
    0%, 100% {
        opacity: 0.4;
        filter: brightness(1) drop-shadow(0 0 0 transparent);
    }
    50% {
        opacity: 0.8;
        filter: brightness(1.5) drop-shadow(0 0 10px rgba(255,255,255,0.8));
    }
}

/* Petal yang sedang di-hover */
#petalContainer div:hover {
    animation-play-state: paused;
    transform: scale(1.5) !important;
    transition: transform 0.3s ease !important;
}

/* Responsive untuk mobile */
@media (max-width: 768px) {
    #petalContainer div {
        font-size: 16px !important;
    }
}
</style>
"""

display(HTML(petal_effects))

# Tambahkan musik background yang SIMPLE
music_control = """
<div style="position:fixed; bottom:15px; right:15px; z-index:10000;">
    <div style="background: rgba(255, 255, 255, 0.9); padding: 10px 15px; border-radius: 25px; box-shadow: 0 5px 15px rgba(0,0,0,0.1); display: flex; align-items: center; gap: 10px; backdrop-filter: blur(5px);">
        <span style="color: #9b59b6; font-family: 'Poppins', sans-serif; font-size: 14px;">🎵 Musik:</span>
        <button onclick="toggleMusic()" id="musicBtn" style="background: #9b59b6; color: white; border: none; border-radius: 50%; width: 36px; height: 36px; cursor: pointer; font-size: 16px; transition: all 0.3s ease;">▶️</button>
    </div>
</div>

<script>
let musicPlaying = false;
let audio = null;

function toggleMusic() {
    const btn = document.getElementById('musicBtn');

    if (!musicPlaying) {
        // Coba buat audio sederhana (tanpa autoplay)
        try {
            audio = new Audio('data:audio/mp3;base64,SUQzBAAAAAAAI1RTU0UAAAAPAAADTGF2ZjU4Ljc2LjEwMAAAAAAAAAAAAAAA//tQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAWGluZwAAAA8AAAAHAAACcQCAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAASW5mbwAAAA8AAAAIAAACIECAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgAAAAAAAABJTkZPAAAAEAAAACAAABJTICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAg');
            audio.loop = true;
            audio.volume = 0.2;

            // Coba play dengan user interaction
            const playPromise = audio.play();

            if (playPromise !== undefined) {
                playPromise.then(() => {
                    musicPlaying = true;
                    btn.innerHTML = '⏸️';
                    btn.style.background = '#e84393';
                }).catch(error => {
                    console.log('Autoplay prevented:', error);
                    // Fallback: tampilkan pesan
                    btn.innerHTML = '🔇';
                    btn.title = 'Klik untuk enable audio';
                });
            }
        } catch(e) {
            console.log('Audio error:', e);
            btn.innerHTML = '❌';
        }
    } else {
        if (audio) {
            audio.pause();
            audio.currentTime = 0;
        }
        musicPlaying = false;
        btn.innerHTML = '▶️';
        btn.style.background = '#9b59b6';
    }
}

// Coba autoplay dengan interaksi pertama
document.addEventListener('click', function initAudio() {
    if (!musicPlaying) {
        // Hanya coba sekali
        document.removeEventListener('click', initAudio);
        toggleMusic();
    }
}, { once: true });
</script>
"""

display(HTML(music_control))

# Tambahkan tombol refresh petal
refresh_button = """
<div style="position:fixed; bottom:15px; left:15px; z-index:10000;">
    <button onclick="refreshPetals()" style="background: rgba(255, 255, 255, 0.9); color: #9b59b6; border: 2px solid #9b59b6; border-radius: 25px; padding: 8px 16px; cursor: pointer; font-family: 'Poppins', sans-serif; font-size: 14px; backdrop-filter: blur(5px); transition: all 0.3s ease;">
        🌸 Refresh Petal
    </button>
</div>

<script>
function refreshPetals() {
    const container = document.getElementById('petalContainer');
    if (container) {
        // Hapus semua petal
        while (container.firstChild) {
            container.firstChild.remove();
        }
        // Buat petal baru
        startPetalEffect();

        // Feedback visual
        const btn = event.target;
        btn.innerHTML = '✅ Refreshed!';
        btn.style.background = '#9b59b6';
        btn.style.color = 'white';

        setTimeout(() => {
            btn.innerHTML = '🌸 Refresh Petal';
            btn.style.background = 'rgba(255, 255, 255, 0.9)';
            btn.style.color = '#9b59b6';
        }, 1500);
    }
}
</script>
"""

display(HTML(refresh_button))

<>:688: SyntaxWarning: invalid escape sequence '\S'
<>:688: SyntaxWarning: invalid escape sequence '\S'
/tmp/ipython-input-1591781483.py:688: SyntaxWarning: invalid escape sequence '\S'
  const animationName = keyframes.match(/@keyframes (\S+)/)[1];
